# Summary analysis: Kinetic parameters 4B

## Jump Distance on Knot trajectory

### speed = 0.7 px / frame



In [ ]:
%config InteractiveShellApp.pylab_import_all = False
%matplotlib inline
%pylab inline
%reload_ext autoreload
%autoreload 2

import sys
import os
import fnmatch

import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
from pandas import DataFrame, Series
import seaborn as sns
import glob
from datetime import datetime
from scipy import stats

propsFn='props.csv'
props=pd.read_csv(propsFn, header=None, index_col=0, squeeze=True,delim_whitespace=True).to_dict()

base=props['BaseDir']
expFile=props['allExpFn']

RawDataDir = os.path.join(base,props['RawDataDir'])+'\\'
ProcessingDir = os.path.join(base,props['ProcessingDir'])+'\\Fig2FG\\'
outputDir = os.path.join(base,props['outputDir'])+'\\'

if not os.path.isdir(ProcessingDir):
    os.makedirs(ProcessingDir)
if not os.path.isdir(outputDir):
    os.makedirs(outputDir)

os.chdir('..\\')
import functions.matrixUtilities_joh as mu
import matplotlib.pyplot as plt
import models.experiment as xp
import models.experiment_set as es
import functions.paperFigureProps as pfp
pfp.paper()
inToCm=2.54


In [ ]:
info=pd.read_csv(expFile, sep=',')#pd.read_csv(expFile,quotechar='"', sep=',', converters={'bdGroup':ast.literal_eval})
info=info[info.stimulusProtocol=='4b']
info

In [ ]:
# collect meta information and save to new csv file for batch processing

aviPath=[]
posPath=[]
PLPath=[]
expTime = []
    
for index,row in info.iterrows():
    startDir=RawDataDir+row.path+'\\'
    #startDir='D:\\data\\b\\2017\\'+row.path+'\\'
    #if not os.path.isdir(startDir):
    #    startDir='E:\\b\\2017\\'+row.path+'\\'
        
    posPath.append(glob.glob(startDir+'PositionTxt*.txt')[0])
    PLPath.append(glob.glob(startDir+'PL*.txt')[0])
    
    head, tail = os.path.split(posPath[-1])
    currTime=datetime.strptime(tail[-23:-4], '%Y-%m-%dT%H_%M_%S')
    expTime.append(currTime)
    
info['txtPath']=posPath
info['pairList']=PLPath

info['epiDur'] = 5      # duration of individual episodes (default: 5 minutes)
info['episodes'] = -1   # number of episodes to process: -1 to load all episodes (default: -1)
info['inDish'] = 10#np.arange(len(posPath))*120     # time in dish before experiments started (default: 10)
info['arenaDiameter_mm'] = 100 # arena diameter (default: 100 mm)
info['minShift'] = 60 # minimum number of seconds to shift for control IAD
info['episodePLcode'] = 0 # flag if first two characters of episode name encode animal pair matrix (default: 0)
info['recomputeAnimalSize'] = 0 # flag to compute animals size from avi file (takes time, default: 1)
info['SaveNeighborhoodMaps'] = 0 # flag to save neighborhood maps for subsequent analysis (takes time, default: 1)
info['computeLeadership'] = 0 # flag to compute leadership index (takes time, default: 1)
info['ComputeBouts'] = 1 # flag to compute swim bout frequency (takes time, default: 1)
info['set'] = np.arange(len(posPath))   # experiment set: can label groups of experiments (default: 0)
info['ProcessingDir']=ProcessingDir
info['outputDir']=outputDir

info['expTime']=expTime

csvFile=os.path.join(ProcessingDir,'Fig2_FG.csv')
info.to_csv(csvFile,encoding='utf-8')
info

In [ ]:
def readExperiment(keepData=False):
    tmp=es.experiment_set(csvFile=csvFile)
    if keepData:
        return tmp
    else:
        return 1

expSet=readExperiment(keepData=True)

In [ ]:
csvPath = []
for f in [mu.splitall(x)[-1][:-4] for x in info.txtPath]:
    csvPath.append(glob.glob(ProcessingDir+f+'*siSummary*.csv')[0])

df=pd.DataFrame()
i=0
for fn in csvPath:
    print(fn)
    tmp=pd.read_csv(fn,index_col=0,sep=',')
    tmp.animalSet=i
    tmp.animalIndex=tmp.animalIndex+((i)*15)
    df=pd.concat([df,tmp])
    i+=1
df['episode']=[x.strip().replace('_','') for x in df['episode']]

print('df shape',df.shape)

In [ ]:
d=df.time
r=datetime(2017,1,1)
t2=[pd.to_datetime(x).replace(day=1,month=1)for x in df.time]
t3=[(x-r)/pd.Timedelta('1 hour') for x in t2]
df['t2']=t2
df['t3']=t3
df

## Habituation or Fatigue within 20 hours?

Plot shoaling index during closed loop skype episodes over time.

In [ ]:
sns.tsplot(data=df, time="inDishTime",value="si",unit="animalIndex",condition="episode",estimator=np.nanmean,interpolate=False,err_style="ci_bars");
plt.xlim([0,8*60])

In [ ]:
idx=(df['inDishTime']<350) & (df['inDishTime']>45)
episodeNames=df['episode'].unique()
dfDR=df[idx]
dfDR.loc[:,'ag']=0
dfDR.loc[dfDR.age>16,'ag']=1
dfDR.loc[dfDR.age>21,'ag']=2

In [ ]:
tmp=dfDR.groupby(['episode','animalIndex'],sort=True)[['si']]
xax=np.arange(episodeNames.shape[0])

err=tmp.std().unstack().values.T

fig, axes = plt.subplots(figsize=(10, 7))

tmp=tmp.mean().unstack()
tmp['xax']=xax

axes=tmp.plot(x='xax',kind='line',marker='o',yerr=err,
                                  linestyle=':',ax=axes,legend=False)

dfSkype=df[idx]
dfSkype=dfSkype[dfSkype['episode']==episodeNames[-1]]
mn=dfSkype.si.mean()
er=dfSkype.si.std()




axes.set_ylabel('attraction index')
plt.xlim([0,xax.max()+xax.max()*0.1])
axes.axhline(0,ls=':',color='k')
axes.set_title('Individual Pair Disc Size Tuning, Night vs. Day, n=8 pairs');

In [ ]:
tmp=dfDR.groupby(['episode','age'],sort=True)[['si']]
xax=np.arange(episodeNames.shape[0])

err=tmp.std().unstack().values.T
mndf=tmp.mean().unstack().reset_index()
mndf['xax']=xax
fig, axes = plt.subplots(figsize=(10, 7))





axes=mndf.plot(x='xax',y='si',kind='line',marker='o',yerr=0,
                                  linestyle=':',ax=axes,legend=True)


axes.set_ylabel('attraction index')
plt.xlim([0,xax.max()+xax.max()*0.1])
axes.axhline(0,ls=':',color='k')
axes.set_title('Individual Pair Disc Size Tuning, Night vs. Day, n=8 pairs');


In [ ]:
tmp_g=dfDR.groupby(['episode','ag'],sort=True)[['si']]

err_g=tmp_g.std().unstack().values.T
mndf_g=tmp_g.mean().unstack().reset_index()
fig, axes = plt.subplots(figsize=(10, 7))

mndf_g['xax']=xax
axes=mndf_g.plot(x='xax',y='si',kind='line',marker='o',yerr=0,
                                  linestyle=':',ax=axes,legend=True)



axes.set_ylabel('attraction index')
plt.xlim([0,xax.max()+xax.max()*0.1])
axes.axhline(0,ls=':',color='k')
axes.set_title('Individual Pair Disc Size Tuning, Night vs. Day, n=8 pairs');


In [ ]:
sns.set_palette('viridis',6)
co=sns.color_palette("viridis", 6)

col=[co[0],co[1]]
pfp.paper()
inToCm=2.54

fig, axes = plt.subplots(nrows=1, ncols=2, sharex=True, sharey=True,figsize=(9/inToCm,4.5/inToCm))


mean=mndf.values[:7,1:-1]
sd=err.T[:7,:]
xax=np.array([1/4., 1/2., 1., 1.5, 2., 3., 4.])*5.7
#xax=np.round(xax)
i=0
axes[i].axhline(0,ls=':',color='gray')
axes[i].plot(xax,mean)
dfx=pd.DataFrame(mean,columns=df.age.unique())
dfx['xax']=xax[:8]
ax=dfx.plot(kind='line',marker='o',ls='',x='xax',
                 ax=axes[i],color=co,ms=5,
                 markeredgecolor='k',markeredgewidth=1,logx=True,legend=False)

i=1
mean=mndf.values[7:,1:-1]
sd=err.T[7:,:]
#xax=np.array([1/4., 1/2., 1., 1.5, 2., 3., 4.])

axes[i].axhline(0,ls=':',color='gray')
axes[i].plot(xax,mean)
dfx=pd.DataFrame(mean,columns=df.age.unique()[::-1])
dfx['xax']=xax[:8]
ax=dfx.plot(kind='line',marker='o',ls='',x='xax',
                 ax=axes[i],color=co,ms=5,
                 markeredgecolor='k',markeredgewidth=1,logx=True,legend=False)

axes[0].set_xticks([1,2,5,10,20]);
xlab='%i %i %i %i %i' % tuple([1,2,5,10,20])
axes[0].set_xticklabels(xlab.split());
axes[0].set_yticks([0,.2,.4]);
axes[0].set_xlim([1,28])
axes[0].set_ylim([-.05,.45]);
axes[0].set_xlabel('Speed (mm/sec)')
axes[1].set_xlabel('Speed (mm/sec)')
axes[0].set_title('Intermittent motion')
axes[1].set_title('Constant motion')
axes[0].set_ylabel('Attraction')
#axes[0].text(1,-0.09,'1x=5.7 mm/sec')
#axes[1].text(1,-0.09,'1x=5.7 mm/sec')
#fig.tight_layout()
sns.despine()
fig.subplots_adjust(top=0.75)
#plt.legend(title="age (dpf)")


In [ ]:
pfp.paper()
sns.set_palette('viridis',3)

inToCm=2.54

fig, axes = plt.subplots(nrows=1, ncols=2, sharex=True, sharey=True,figsize=(9/inToCm,4.5/inToCm))


mean=mndf_g.values[:7,1:-1]
sd=err_g.T[:7,:]
xax=np.array([1/4., 1/2., 1., 1.5, 2., 3., 4.])*5.7
#xax=np.round(xax)
i=0
axes[i].axhline(0,ls=':',color='gray')
axes[i].plot(xax,mean)
dfx=pd.DataFrame(mean,columns=dfDR.ag.unique())
dfx['xax']=xax[:8]
ax=dfx.plot(kind='line',marker='o',ls='',x='xax',
                 ax=axes[i],ms=5,
                 markeredgecolor='k',markeredgewidth=1,logx=True,legend=False)

i=1
mean=mndf_g.values[7:,1:-1]
sd=err_g.T[7:,:]

axes[i].axhline(0,ls=':',color='gray')
axes[i].plot(xax,mean)
dfx=pd.DataFrame(mean,columns=dfDR.ag.unique()[::-1])
dfx['xax']=xax[:8]
ax=dfx.plot(kind='line',marker='o',ls='',x='xax',
                 ax=axes[i],ms=5,
                 markeredgecolor='k',markeredgewidth=1,logx=True,legend=False)

axes[0].set_xticks([1,2,5,10,20]);
xlab='%i %i %i %i %i' % tuple([1,2,5,10,20])
axes[0].set_xticklabels(xlab.split());
axes[0].set_yticks([0,.2,.4]);
axes[0].set_xlim([1,28])
axes[0].set_ylim([-.05,.45]);
axes[0].set_xlabel('Speed (mm/sec)')
axes[1].set_xlabel('Speed (mm/sec)')
axes[0].set_title('Intermittent motion')
axes[1].set_title('Constant motion')
axes[0].set_ylabel('Attraction')

sns.despine()
fig.subplots_adjust(top=0.75)

L=plt.legend(bbox_to_anchor=(.5, 1), loc=2, borderaxespad=0.,handletextpad=0,title="Age (dpf)")
#L=plt.legend(ncol=1, loc='upper right',)

L.get_texts()[0].set_text('<17')
L.get_texts()[1].set_text('17-21')
L.get_texts()[2].set_text('>21')

figPath=outputDir+'\\2F_speedBoutVsConstan.svg'
plt.savefig(figPath)

In [ ]:
inToCm=2.54

#fig, axes = plt.subplots(nrows=1, ncols=2, sharex=True, sharey=True,figsize=(9/inToCm,4.5/inToCm))

fig, axes = plt.subplots(sharex=True, sharey=True,figsize=(4.5/inToCm,4.5/inToCm))


co=sns.color_palette("Dark2", 8)
mean=mndf.values[:7,1:-1]
dfx=pd.DataFrame(mean,columns=np.sort(df.age.unique()))
#dfx.sort_index(axis=1,inplace=True)
sns.heatmap(dfx.values.astype('float').T,center=0,cmap='seismic',ax=axes)
plt.yticks(range(8),np.sort(df.age.unique()));
dfx.loc[-1]=np.zeros(dfx.shape[1])
dfx.index = dfx.index + 1  # shifting index
dfx = dfx.sort_index()  # sorting by index
#sns.heatmap(dfx,center=0,cmap='bwr')

In [ ]:
dfx

In [ ]:
maxPos=[]
fig, ax = plt.subplots(nrows=8, ncols=1, sharex=True, sharey=True,figsize=(4.5/inToCm,25/inToCm))
x=np.array([0,0.25,0.5,1,1.5,2,3])
for i in range(8):
    y=dfx.values[:-1,i]
    z=np.polyfit(x,y,4)
    p = np.poly1d(z)
    xp = np.linspace(0, 3, 1000)
    maxPos.append(np.argmax(p(xp)[:500])/(1000/3.))
    ax[i].plot(x, y, '.', xp, p(xp), '-')
    ax[i].set_ylim([-.10,.4])
    ax[i].axvline(maxPos[-1])

In [ ]:
import scipy.stats
pfp.paper()
inToCm=2.54
plt.figure(figsize=(4.5/inToCm,4.5/inToCm))
ax = plt.gca()

xs=dfx.columns
ys=np.array(maxPos)*5.7
s,i,r,p,std=scipy.stats.linregress(xs,ys)
t=np.linspace(10,30,100)
l=i+s*t
ax.plot(t,l,':',xs,ys,'.')
ax.text(12,6.5,'R: '+str(r)[:4])
#ax.text(12,6,'p: '+str(p)[:3]+str(p)[-4:])
ax.text(12,6,"p = {:.1e}".format(p))

ax.set_ylabel('Vmax [mm/sec]')
ax.set_xlabel('age (dpf)')
sns.despine()

# Individual animal analysis



In [ ]:
dfDR=df[idx]
tmp=dfDR.groupby(['episode','animalIndex'],sort=True)[['si','age','anSize']]
mn=tmp.mean().si.unstack().reset_index()

mean=mn.values[:7,1:-1]
dfx=mn.loc[:6].copy()
dfx=dfx.drop('episode',axis=1)

sns.heatmap(dfx.values.astype('float').T,center=0,cmap='seismic')

dfx.loc[-1]=np.zeros(dfx.shape[1])
dfx.index = dfx.index + 1  # shifting index
dfx = dfx.sort_index()  # sorting by index


In [ ]:
maxPosAll=[]
ageAll=[]
x=np.array([0,0.25,0.5,1,1.5,2,3])
for i in range(dfx.shape[1]):
    y=dfx.values[:-1,i]
    z=np.polyfit(x,y,4)
    p = np.poly1d(z)
    xp = np.linspace(0, 3, 1000)
    if np.max(p(xp)[:400])>.03:
        maxPosAll.append(np.argmax(p(xp)[:400])/(1000/3.)*5.7)
    else:
        maxPosAll.append(np.nan)
    ageAll.append(df[df.animalIndex==dfx.columns[i]].age.values[0])

In [ ]:
mpa=pd.DataFrame({'age':ageAll,'mp':maxPosAll})
mpa.head()

In [ ]:
sns.jointplot(mpa.age,mpa.mp)

In [ ]:
maxPosIndMn=mpa.groupby(['age']).mean().mp
maxPosIndSTD=mpa.groupby(['age']).std().mp
maxPosIndSTD

In [ ]:
import scipy.stats
pfp.paper()
inToCm=2.54
plt.figure(figsize=(4.5/inToCm,4.5/inToCm))
ax = plt.gca()
imaxCol='gray'

xs=maxPosIndMn.index.values
ys=maxPosIndMn.values
s,i,r,p,std=scipy.stats.linregress(xs,ys)

bidx=(0<np.ones(dfDR.shape[0]))#&(dfDR.episode=='01js1o4s')
swimmSpeed=dfDR[bidx].groupby('age').mean()['avgSpeed_smooth'].reset_index()
x=swimmSpeed.age.values
y=swimmSpeed.avgSpeed_smooth.values
so,io,ro,po,stdo=scipy.stats.linregress(x,y)


t=np.linspace(10,30,100)
l=i+s*t
l2=io+so*t

ax.plot(t,l,'--',xs,ys,'.',color=imaxCol,markersize=20)
ax.plot(t,l2,'--',color='k')
ax.plot(x,y,'.',color='k')
(_, caps, _)=ax.errorbar(xs,ys,maxPosIndSTD.values,ls='',color=imaxCol)
for cap in caps:
    cap.set_markeredgewidth(1)
    
ax.text(11,7.5,'R: '+str(ro)[:4],color='k',fontsize=10)
ax.text(11,6.7,"p = {:.1e}".format(po),color='k',fontsize=10)
ax.text(20,2,'R: '+str(r)[:4],color=imaxCol,fontsize=10)
ax.text(20,1.2,"p = {:.1e}".format(p),color=imaxCol,fontsize=10)

ax.text(11,9.5,'Preferred stimulus speed',color=imaxCol,fontsize=10)
ax.text(11,8.7,'Own swim speed',color='k',fontsize=10)

ax.set_ylabel('Speed (mm/sec)')
ax.set_xlabel('Age (dpf)')
plt.ylim([1,8])
sns.despine()
figPath=outputDir+'\\2G_Speed_corr.svg'
plt.savefig(figPath)

from shutil import copy2

def splitall(path):
    allparts = []
    while 1:
        parts = os.path.split(path)
        if parts[0] == path:  # sentinel for absolute paths
            allparts.insert(0, parts[0])
            break
        elif parts[1] == path: # sentinel for relative paths
            allparts.insert(0, parts[1])
            break
        else:
            path = parts[0]
            allparts.insert(0, parts[1])
    return allparts



for i,row in info.iterrows():
    fn=row.txtPath
    head, tail = os.path.split(fn)

    copyList=[]
    copyList.append(glob.glob(head+'\\ROI*.csv')[0])
    copyList.append(glob.glob(head+'\\PositionTxt*.txt')[0])
    copyList.append(glob.glob(head+'\\PL*.txt')[0])
    copyList.append(glob.glob(head+'\\*anSize.csv')[0])
    
    for f in copyList:
        print(f)
        if f[0]=='E':
            keepSlash=3
        else:
            keepSlash=4
        toDirectory = "e:\\b\\LarschAndBaier2018\\RawData\\" + os.path.join(*splitall(f)[keepSlash:-1])+"\\"
        #toDirectory = "e:\\b\\LarschAndBaier2018\\RawData\\" 
        if not os.path.isdir(toDirectory):
            os.makedirs(toDirectory)
        
        copy2(f, toDirectory)
